In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import requests
import zipfile
import io


### Fase 1: Extracción

In [2]:
print('\n--- FASE 1: Extracción ---')

df = pd.read_csv('2025-01.csv')


--- FASE 1: Extracción ---


### Fase 2: Transmoración

In [3]:
print('\n--- FASE 2: Transformación ---')

# 2.1 --- Limpieza de datos ---
print('\nIniciando limpieza de datos ...')

# Mostrar las primeras filas del dataset
df.head()

# Mostrar informacio del dataset
df.info()
df.describe()

# Mostrar el tipo de datos
df.dtypes

# Mostrar el tamaño del dataset
df.shape


--- FASE 2: Transformación ---

Iniciando limpieza de datos ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1809775 entries, 0 to 1809774
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Genero_Usuario         object 
 1   Edad_Usuario           float64
 2   Bici                   int64  
 3   Ciclo_Estacion_Retiro  object 
 4   Fecha_Retiro           object 
 5   Hora_Retiro            object 
 6   Ciclo_EstacionArribo   object 
 7   Fecha_Arribo           object 
 8   Hora_Arribo            object 
dtypes: float64(1), int64(1), object(7)
memory usage: 124.3+ MB


(1809775, 9)

In [ ]:
df.head(13)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_EstacionArribo,Fecha_Arribo,Hora_Arribo
0,M,26.0,5180930,568,2024-12-31,23:57:02,572,01/01/2025,00:00:03
1,F,54.0,3653953,283,2024-12-31,23:51:40,596,01/01/2025,00:00:41
2,M,38.0,7511322,034,2024-12-31,23:48:36,064,01/01/2025,00:00:59
3,M,41.0,3804572,258,2024-12-31,23:54:11,023,01/01/2025,00:01:08
4,M,35.0,3848405,043,2024-12-31,23:35:28,126,01/01/2025,00:01:17
5,M,24.0,7579436,222,2024-12-31,23:34:33,483,01/01/2025,00:01:58
6,M,31.0,7161920,034,2024-12-31,23:25:47,688,01/01/2025,00:02:16
7,M,22.0,7220593,254,2024-12-31,23:50:20,539,01/01/2025,00:02:41
8,M,29.0,4031258,560,2024-12-31,23:52:11,120,01/01/2025,00:03:28
9,F,24.0,6463376,560,2024-12-31,23:51:50,120,01/01/2025,00:03:31


In [5]:
# Mostrar el tipo de datos antes de la transformación
print('Tipos de datos antes de la transformación a datetime: ')
print(df[['Fecha_Retiro', 'Fecha_Arribo']].dtypes)

# Convertir columnas de fecha a formato datetime
df['Fecha_Retiro'] = pd.to_datetime(df['Fecha_Retiro'], dayfirst=True, errors='coerce')
df['Fecha_Arribo'] = pd.to_datetime(df['Fecha_Arribo'], dayfirst=True, errors='coerce')

Tipos de datos antes de la transformación a datetime: 
Fecha_Retiro    object
Fecha_Arribo    object
dtype: object


In [6]:
df.shape

(1809775, 9)

In [9]:
print('Mostrar tipos de datos después de la transformación')
print(df[['Fecha_Retiro', 'Fecha_Arribo']].dtypes)

Mostrar tipos de datos después de la transformación
Fecha_Retiro    datetime64[ns]
Fecha_Arribo    datetime64[ns]
dtype: object


### Fase 2: Feature engineering

In [10]:
# === Continuación del ETL (a partir de aquí) ===

print('\n--- FASE 2 (cont.): Feature engineering y preparación ---')

# 1) Verificación de columnas esperadas
cols_esperadas = [
    'Genero_Usuario','Edad_Usuario','Bici',
    'Ciclo_Estacion_Retiro','Fecha_Retiro','Hora_Retiro',
    'Ciclo_EstacionArribo','Fecha_Arribo','Hora_Arribo'
]
faltantes = [c for c in cols_esperadas if c not in df.columns]
if faltantes:
    raise ValueError(f'Faltan columnas en el CSV: {faltantes}')

# 2) Unir Fecha + Hora a datetimes nuevos
import pandas as pd
def _combinar_fecha_hora(fecha_series, hora_series):
    f = pd.to_datetime(fecha_series, errors='coerce')
    h = pd.to_datetime(hora_series, format='%H:%M:%S', errors='coerce')
    # si no, intentamos HH:MM
    mask_na = h.isna()
    if mask_na.any():
        h_alt = pd.to_datetime(hora_series[mask_na], format='%H:%M', errors='coerce')
        h.loc[mask_na] = h_alt
    return (
        f
        + pd.to_timedelta(h.dt.hour.fillna(0).astype('Int64'), unit='h')
        + pd.to_timedelta(h.dt.minute.fillna(0).astype('Int64'), unit='m')
        + pd.to_timedelta(h.dt.second.fillna(0).astype('Int64'), unit='s')
    )

df['fecha_origen_recorrido']  = _combinar_fecha_hora(df['Fecha_Retiro'], df['Hora_Retiro'])
df['fecha_destino_recorrido'] = _combinar_fecha_hora(df['Fecha_Arribo'], df['Hora_Arribo'])

# 3) Nulos críticos: quitar filas sin fechas combinadas válidas
antes = df.shape[0]
df = df.dropna(subset=['fecha_origen_recorrido','fecha_destino_recorrido']).copy()
print(f'Filas eliminadas por fechas combinadas nulas: {antes - df.shape[0]}')

# 4) Duración en segundos y limpieza de negativos
df['duracion_recorrido'] = (df['fecha_destino_recorrido'] - df['fecha_origen_recorrido']).dt.total_seconds()
neg = (df['duracion_recorrido'] < 0).sum()
if neg:
    print(f'Aviso: {neg} duraciones negativas ajustadas a 0 (posible error de captura)')
df['duracion_recorrido'] = df['duracion_recorrido'].fillna(0).clip(lower=0)

# 5) One-Hot de género SIN normalizar (solo M/F)
#    Creamos columnas binarias garantizadas, aunque una categoría no aparezca en el mes.
df['genero_M'] = (df['Genero_Usuario'] == 'M').astype(int)
df['genero_F'] = (df['Genero_Usuario'] == 'F').astype(int)

# 6) Discretización de duración (como en tu BA)
bins   = [0, 1200, 2400, 3600, df['duracion_recorrido'].max()]
labels = ['Viaje Corto', 'Viaje Mediano', 'Viaje Largo', 'Viaje Muy Largo (Anomalía)']
df['categoria_duracion'] = pd.cut(df['duracion_recorrido'], bins=bins, labels=labels, right=False, include_lowest=True)

# 7) Normalización MinMax SOLO de duración (no tocamos género, bici, etc.)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['duracion_recorrido_norm'] = scaler.fit_transform(df[['duracion_recorrido']])

# 8) Otras features de calendario
df['duracion_minutos'] = df['duracion_recorrido'] / 60.0
df['dia_semana']       = df['fecha_origen_recorrido'].dt.dayofweek  # 0=Lun .. 6=Dom
df['hora_inicio']      = df['fecha_origen_recorrido'].dt.hour
df['tipo_dia']         = np.where(df['dia_semana'] >= 5, 'Fin de Semana', 'Entre Semana')

# 9) Renombres útiles (solo alias para facilitar análisis, no borro las columnas originales)
df['nombre_estacion_origen']  = df['Ciclo_Estacion_Retiro'].astype(str)
df['nombre_estacion_destino'] = df['Ciclo_EstacionArribo'].astype(str)

# 10) Vista rápida
print('\nConteo por categoría de duración:')
print(df['categoria_duracion'].value_counts(dropna=False))

print('\nResumen de duración (min):')
print(df['duracion_minutos'].describe())

print('\nPreview transformado:')
display(df[['Bici','Edad_Usuario',
            'nombre_estacion_origen','nombre_estacion_destino',
            'fecha_origen_recorrido','fecha_destino_recorrido',
            'duracion_recorrido','duracion_recorrido_norm','duracion_minutos',
            'categoria_duracion','dia_semana','hora_inicio','tipo_dia',
            'genero_M','genero_F']].head(20))

# === FASE 3: Carga ===
print('\n--- FASE 3: Carga ---')
df_salida_cols = [
    'Bici','Edad_Usuario',
    'nombre_estacion_origen','nombre_estacion_destino',
    'fecha_origen_recorrido','fecha_destino_recorrido',
    'duracion_recorrido','duracion_recorrido_norm','duracion_minutos',
    'categoria_duracion','dia_semana','hora_inicio','tipo_dia',
    'genero_M','genero_F'
]
df_salida_cols = [c for c in df_salida_cols if c in df.columns]
df_final = df[df_salida_cols].copy()

csv_out  = 'viajes_cdmx_limpios.csv'
parq_out = 'viajes_cdmx_limpios.parquet'

df_final.to_csv(csv_out, index=False)
print(f'CSV guardado: {csv_out}')

try:
    df_final.to_parquet(parq_out, index=False)
    print(f'Parquet guardado: {parq_out}')
except ImportError:
    print("Para Parquet, instala pyarrow: pip install pyarrow")



--- FASE 2 (cont.): Feature engineering y preparación ---
Filas eliminadas por fechas combinadas nulas: 0

Conteo por categoría de duración:
categoria_duracion
Viaje Corto                   1364940
Viaje Mediano                  391597
Viaje Largo                     46210
Viaje Muy Largo (Anomalía)       7027
NaN                                 1
Name: count, dtype: int64

Resumen de duración (min):
count    1.809775e+06
mean     1.593460e+01
std      7.927174e+02
min      3.166667e-01
25%      7.100000e+00
50%      1.190000e+01
75%      1.980000e+01
max      1.032525e+06
Name: duracion_minutos, dtype: float64

Preview transformado:


,Bici,Edad_Usuario,nombre_estacion_origen,nombre_estacion_destino,fecha_origen_recorrido,fecha_destino_recorrido,duracion_recorrido,duracion_recorrido_norm,duracion_minutos,categoria_duracion,dia_semana,hora_inicio,tipo_dia,genero_M,genero_F
0,5180930,26.0,568,572,2024-12-31 23:57:02,2025-01-01 00:00:03,181.0,0.000003,3.016667,Viaje Corto,1,23,Entre Semana,1,0
1,3653953,54.0,283,596,2024-12-31 23:51:40,2025-01-01 00:00:41,541.0,0.000008,9.016667,Viaje Corto,1,23,Entre Semana,0,1
2,7511322,38.0,034,064,2024-12-31 23:48:36,2025-01-01 00:00:59,743.0,0.000012,12.383333,Viaje Corto,1,23,Entre Semana,1,0
3,3804572,41.0,258,023,2024-12-31 23:54:11,2025-01-01 00:01:08,417.0,0.000006,6.950000,Viaje Corto,1,23,Entre Semana,1,0
4,3848405,35.0,043,126,2024-12-31 23:35:28,2025-01-01 00:01:17,1549.0,0.000025,25.816667,Viaje Mediano,1,23,Entre Semana,1,0
5,7579436,24.0,222,483,2024-12-31 23:34:33,2025-01-01 00:01:58,1645.0,0.000026,27.416667,Viaje Mediano,1,23,Entre Semana,1,0
6,7161920,31.0,034,688,2024-12-31 23:25:47,2025-01-01 00:02:16,2189.0,0.000035,36.483333,Viaje Mediano,1,23,Entre Semana,1,0
7,7220593,22.0,254,539,2024-12-31 23:50:20,2025-01-01 00:02:41,741.0,0.000012,12.350000,Viaje Corto,1,23,Entre Semana,1,0
8,4031258,29.0,560,120,2024-12-31 23:52:11,2025-01-01 00:03:28,677.0,0.000011,11.283333,Viaje Corto,1,23,Entre Semana,1,0
9,6463376,24.0,560,120,2024-12-31 23:51:50,2025-01-01 00:03:31,701.0,0.000011,11.683333,Viaje Corto,1,23,Entre Semana,0,1



--- FASE 3: Carga ---
CSV guardado: viajes_cdmx_limpios.csv
Parquet guardado: viajes_cdmx_limpios.parquet


### Fase 3: Medidas de tendencia central

In [11]:
print("\n--- FASE 3: Medidas de tendencia central ---")

# Seleccionamos solo las columnas numéricas
numeric_cols = df_final.select_dtypes(include=[np.number]).columns
print(f"Columnas numéricas: {list(numeric_cols)}\n")

# Calculamos media, mediana y moda
medidas = pd.DataFrame({
    "Media":   df_final[numeric_cols].mean(),
    "Mediana": df_final[numeric_cols].median(),
    "Moda":    df_final[numeric_cols].mode().iloc[0]  # tomamos la primera moda
})

print("Medidas de tendencia central (numéricas):\n")
display(medidas)

# Guardar también en CSV si quieres entregar
medidas.to_csv("medidas_tendencia_central.csv")
print("Archivo 'medidas_tendencia_central.csv' guardado.")



--- FASE 3: Medidas de tendencia central ---
Columnas numéricas: ['Bici', 'Edad_Usuario', 'duracion_recorrido', 'duracion_recorrido_norm', 'duracion_minutos', 'dia_semana', 'hora_inicio', 'genero_M', 'genero_F']

Medidas de tendencia central (numéricas):



,Media,Mediana,Moda
Bici,5.456875e+06,5.452651e+06,8.537367e+06
Edad_Usuario,3.400352e+01,3.200000e+01,3.000000e+01
duracion_recorrido,9.560760e+02,7.140000e+02,3.470000e+02
duracion_recorrido_norm,1.512597e-05,1.121846e-05,5.294466e-06
duracion_minutos,1.593460e+01,1.190000e+01,5.783333e+00
dia_semana,2.710566e+00,3.000000e+00,3.000000e+00
hora_inicio,1.415466e+01,1.500000e+01,1.800000e+01
genero_M,6.922932e-01,1.000000e+00,1.000000e+00
genero_F,2.774991e-01,0.000000e+00,0.000000e+00


Archivo 'medidas_tendencia_central.csv' guardado.
